In [1]:
# __define-ocg__  Setup environment
!pip install langchain faiss-cpu sentence-transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 26.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

# Hazır sağlık veri seti
dataset = load_dataset("medical_questions_pairs", split="train")
dataset = dataset.shuffle(seed=42).select(range(200))  # küçük örnek
dataset = dataset.to_pandas()

print("Veri seti örneği:")
print(dataset.head())


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/314k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3048 [00:00<?, ? examples/s]

Veri seti örneği:
   dr_id                                         question_1  \
0      7  I have this patch of peeling/blistering skin o...   
1     10  What is an easy home remedie for premature eja...   
2      5  How often should I get my custom  orthotics  e...   
3      8  Is it possible to ovulate one day after period...   
4     10   What are some illnesses of the lymphatic system?   

                                          question_2  label  
0  I walked barefoot yesterday and my skin on the...      0  
1  Could you apprise me of some easy home remedie...      1  
2  How frequent should I get my orthotics  for fl...      1  
3                      How can I track my ovulation?      0  
4  Could you educated me about some illnesses of ...      1  


In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

# Soruları embedding’e çevir
embeddings = model.encode(dataset["question_1"].tolist())

# FAISS index oluştur
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

print("Vektör index oluşturuldu:", index.ntotal, "soru eklendi.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vektör index oluşturuldu: 200 soru eklendi.


In [4]:
def find_answer(query, k=3):
    query_emb = model.encode([query])
    distances, indices = index.search(np.array(query_emb), k)
    results = dataset.iloc[indices[0]]
    return results["question_1"].tolist()

# Test
user_query = "baş ağrısı neden olur?"
similar = find_answer(user_query)
print("Benzer sorular:")
for s in similar:
    print("-", s)


Benzer sorular:
- I'm having genital herpes outbreak, usuallly I take acyclovior. But pharm told me it might affect my ulcer. Any home remedies I can do to dry this up?!
- Hello I am using neutrogena from 15 days ..Acne is not leaving what should I do?
- Are food intolerances like msg and onion permanent?


In [5]:
from transformers import pipeline

# Daha güçlü model
qa_model = pipeline("text2text-generation", model="google/flan-t5-large")

def rag_chat(query):
    # Benzer soruları getir
    similar_questions = find_answer(query)

    # Bağlamı oluştur
    context = " ".join(similar_questions)

    # Model için istem (prompt)
    prompt = f"""
    The user asked a health question: "{query}".
    Here are some related questions or facts: {context}.
    Based on this, generate a short, clear, and helpful health advice.
    The answer should be informative but not medical advice.
    """

    # Cevap üret
    result = qa_model(prompt, max_new_tokens=200, temperature=0.7)
    return result[0]["generated_text"]

# Test örnekleri
print("Q: What helps with a cold?")
print("A:", rag_chat("What helps with a cold?"))

print("\nQ: What to do for headache?")
print("A:", rag_chat("What to do for headache?"))

print("\nQ: What are the symptoms of flu?")
print("A:", rag_chat("What are the symptoms of flu?"))


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Q: What helps with a cold?
A: If you have body aches, blocked stuffy nose, headaches, pressure in your face and throat tightness and it feels dry for 6 months is it a bad cold?. If you have body aches, blocked stuffy nose, headaches, pressure in your face and throat tightness and it feels dry for 6 months is it a bad cold?. If you have body aches, blocked stuffy nose, headaches, pressure in your face and throat tightness and it feels dry for 6 months is it a bad cold?. If you have body aches, blocked stuffy nose, headaches, pressure in your face and throat tightness and it feels dry for 6 months is it a bad cold?. If you have body aches, blocked stuffy nose, headaches, pressure in your face and throat tightness and it feels dry for 6 months is it a bad cold?. If you have body aches, blocked stuffy

Q: What to do for headache?
A: Take gabapentin for headaches caused by trimethoprim. Take gabapentin for headaches caused by trimethoprim. Take gabapentin for headaches caused by trimethopri

In [7]:
print(rag_chat("how to reduce fever naturally?"))
print(rag_chat("what to do when you have stomach pain?"))
print(rag_chat("what are the symptoms of covid-19?"))


KeyboardInterrupt: 

In [8]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.7 MB/s eta 0:00:00


In [9]:
%%writefile app.py
# __define-ocg__  Streamlit arayüzü - Sağlık Chatbot
import streamlit as st

st.set_page_config(page_title="🩺 Sağlık Danışmanı Chatbot", page_icon="💬")

st.title("🩺 Health Assistant Chatbot (Demo)")
st.write("Ask a health-related question below 👇")

user_query = st.text_input("Your question:")

if user_query:
    st.write("⏳ Thinking...")
    answer = rag_chat(user_query)
    st.subheader("💬 Chatbot's Answer:")
    st.write(answer)


Writing app.py


In [10]:
!pip install gradio


In [11]:
import re

def clean_repetition(text):
    text = re.sub(r'\b([A-Za-zçğıöşüÇĞİÖŞÜ0-9]+)(?:[,\s]+\1){1,}\b', r'\1', text, flags=re.IGNORECASE)
    text = re.sub(r'\s{2,}', ' ', text).strip()
    return text

def rag_chat(query):
    similar_questions = find_answer(query)
    context = " ".join(similar_questions)

    prompt = f"""
    The user asked: "{query}".
    Related health information: {context}.
    Provide a clear, complete, and informative answer in English.
    Write 2–4 sentences. Avoid repeating words.
    """

    result = qa_model(
        prompt,
        max_new_tokens=180,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        no_repeat_ngram_size=3
    )

    text = result[0]["generated_text"]
    text = clean_repetition(text)

    # Eğer yanıt çok kısa geldiyse, modeli biraz daha uzun çalıştır
    if len(text.split()) < 8:
        result = qa_model(
            prompt,
            max_new_tokens=250,
            do_sample=False,
            num_beams=5,
            early_stopping=True
        )
        text = clean_repetition(result[0]["generated_text"])

    return text


In [12]:
# Gradio arayüzünü "güvenli" çalıştırma - hata mesajını kullanıcıya gösterir
import gradio as gr
import traceback

def safe_chat_interface(question):
    try:
        # kesin string olduğundan emin ol
        q = str(question)
        # rag_chat fonksiyonunu çağır
        answer = rag_chat(q)
        # Eğer None döndüyse, kullanıcıya bilgi ver
        if answer is None:
            return "Model None döndü — rag_chat fonksiyonu boş değer döndürüyor."
        return answer
    except Exception as e:
        tb = traceback.format_exc()
        # Hatanın detayını çıktı olarak dönüyoruz, böylece ne yanlış görürüz
        return f"***ERROR***\n{str(e)}\n\nTraceback:\n{tb}"

demo = gr.Interface(
    fn=safe_chat_interface,
    inputs=gr.Textbox(lines=2, placeholder="Ask your health question here..."),
    outputs="text",
    title="🩺 Health Assistant Chatbot (Safe mode)",
    description="If an error occurs, the full traceback will be shown here."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://785ab19fffa089728a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
